# 전처리

## [1] 환경 설정

In [1]:
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from tqdm.auto import tqdm

tqdm.pandas()

## [2] 경로 설정

In [2]:
# 현재 notebook 위치: scr/
BASE_DIR = os.path.abspath(os.path.join(".."))

LABEL_CSV_PATH = os.path.join(BASE_DIR, "data", "gpt4.1_label_1000d.csv")
LABEL_MAP_PATH = os.path.join(BASE_DIR, "data", "label_map.json")
RAW_JSON_DIR = os.path.join(BASE_DIR, "data", "raw", "원본")
OUTPUT_DIR = os.path.join(BASE_DIR, "data", "processed")

os.makedirs(OUTPUT_DIR, exist_ok=True)

## [3] 라벨 파일 로딩

In [3]:
# label_map 로드
with open(LABEL_MAP_PATH, "r", encoding="utf-8") as f:
    label_map = json.load(f)

# 라벨 CSV 로딩
df_label = pd.read_csv(LABEL_CSV_PATH)
print(df_label.columns)  # "사건번호", "라벨링결과" 같은지 확인


Index(['사건번호', '파일명', '라벨링결과'], dtype='object')


## [4] 텍스트 추출 함수

In [5]:
def extract_target_text(full_text: str) -> str:
    party_info = re.search(r"(【원고, 피상고인】.*?【피고, 상고인】)", full_text, re.DOTALL)
    judgement_info = re.search(r"(【주[\s]*문】.*?)(【이[\s]*유】|【이유】)", full_text, re.DOTALL)

    combined = ""
    if party_info:
        combined += party_info.group(1).strip() + "\n"
    if judgement_info:
        combined += judgement_info.group(1).strip()
    return combined.strip()


## [5] 원본 JSON 순회 및 전처리

In [14]:
from collections import defaultdict
from tqdm.auto import tqdm
import os
import json

data_list = []
missing_stats = defaultdict(list)

def extract_target_text_with_fallback(full_text: str, fallback_len: int = 1000) -> str:
    import re
    party_info = re.search(r"(【원고, 피상고인】.*?【피고, 상고인】)", full_text, re.DOTALL)
    judgement_info = re.search(r"(【주[\s]*문】.*?)(【이[\s]*유】|【이유】)", full_text, re.DOTALL)

    combined = ""
    if party_info:
        combined += party_info.group(1).strip() + "\n"
    if judgement_info:
        combined += judgement_info.group(1).strip()

    if combined.strip():
        return combined.strip()
    else:
        return full_text[:fallback_len].strip()  # fallback 사용

# 전처리 시작
for idx, row in tqdm(df_label.iterrows(), total=len(df_label)):
    filename = str(row["파일명"]).strip()
    label_str = row["라벨링결과"].strip()
    label = label_map.get(label_str, None)

    if label is None:
        missing_stats["label_map_error"].append(filename)
        continue

    json_path = os.path.join(RAW_JSON_DIR, filename)
    if not os.path.exists(json_path):
        missing_stats["file_missing"].append(filename)
        continue

    try:
        with open(json_path, "r", encoding="utf-8") as f:
            case_data = json.load(f)
    except Exception as e:
        missing_stats["json_read_error"].append(filename)
        continue

    full_text = case_data.get("판례내용", "")
    if not full_text:
        missing_stats["no 판례내용"].append(filename)
        continue

    trimmed_text = extract_target_text_with_fallback(full_text)
    if trimmed_text.strip() == full_text[:1000].strip():
        missing_stats["regex_fail_fallback"].append(filename)

    data_list.append({
        "text": trimmed_text,
        "label": label
    })

print(f"\n✅ 총 usable 샘플 수: {len(data_list)}\n")

for reason, files in missing_stats.items():
    print(f"❌ {reason}: {len(files)}건")

# regex_fail_fallback 파일 리스트 출력 (최대 10개만 보기)
print("\n❗ fallback 처리된 파일명 리스트 (정규식 실패)")
for f in missing_stats["regex_fail_fallback"][:10]:
    print("-", f)

# 필요하면 전체를 파일로 저장
fallback_list_path = os.path.join(OUTPUT_DIR, "regex_fallback_list.txt")
with open(fallback_list_path, "w", encoding="utf-8") as fw:
    for f in missing_stats["regex_fail_fallback"]:
        fw.write(f + "\n")

print(f"\n📝 fallback 처리 리스트 저장됨: {fallback_list_path}")

  0%|          | 0/843 [00:00<?, ?it/s]


✅ 총 usable 샘플 수: 843

❌ regex_fail_fallback: 3건

❗ fallback 처리된 파일명 리스트 (정규식 실패)
- 213723.json
- 216653.json
- 223575.json

📝 fallback 처리 리스트 저장됨: c:\dev\github\SKN10-FINAL-3Team\data\processed\regex_fallback_list.txt


## [6] 7:2:1 분할

In [16]:
from collections import Counter

label_counts = Counter([x["label"] for x in data_list])
for label_id, count in label_counts.items():
    print(f"라벨 {label_id}: {count}개")

라벨 7: 117개
라벨 4: 209개
라벨 0: 133개
라벨 5: 62개
라벨 2: 151개
라벨 1: 166개
라벨 3: 4개
라벨 6: 1개


In [18]:
# 라벨 3 ("각하"), 라벨 6 ("취하") 제거
filtered_data_list = [x for x in data_list if x["label"] not in [3, 6]]
print(f"제외 후 usable 샘플 수: {len(filtered_data_list)}")

제외 후 usable 샘플 수: 838


In [22]:
from sklearn.model_selection import train_test_split

# test 10%
train_val, test = train_test_split(
    filtered_data_list,
    test_size=0.1,
    random_state=42,
    stratify=[x["label"] for x in filtered_data_list]
)

# train:val = 7:2 → val 비율은 2/9
train, val = train_test_split(
    train_val,
    test_size=2/9,
    random_state=42,
    stratify=[x["label"] for x in train_val]
)

print(f"Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")



Train: 586, Val: 168, Test: 84


In [24]:
from collections import Counter

def print_label_distribution(data, label_map):
    id_to_label = {v: k for k, v in label_map.items()}
    counts = Counter([x["label"] for x in data])
    for label_id, count in sorted(counts.items()):
        label_name = id_to_label.get(label_id, "Unknown")
        print(f"{label_name} ({label_id}): {count}개")

print("\n📊 Train 분포:")
print_label_distribution(train, label_map)
print("\n📊 Val 분포:")
print_label_distribution(val, label_map)
print("\n📊 Test 분포:")
print_label_distribution(test, label_map)


📊 Train 분포:
인용 (0): 93개
일부인용 (1): 116개
기각 (2): 106개
파기환송 (4): 146개
원심유지 (5): 43개
파기환송, 원심유지 (7): 82개

📊 Val 분포:
인용 (0): 27개
일부인용 (1): 33개
기각 (2): 30개
파기환송 (4): 42개
원심유지 (5): 13개
파기환송, 원심유지 (7): 23개

📊 Test 분포:
인용 (0): 13개
일부인용 (1): 17개
기각 (2): 15개
파기환송 (4): 21개
원심유지 (5): 6개
파기환송, 원심유지 (7): 12개


 ## [7] 전처리 결과 저장

In [21]:
with open(os.path.join(OUTPUT_DIR, "train_data.json"), "w", encoding="utf-8") as f:
    json.dump(train, f, ensure_ascii=False, indent=2)

with open(os.path.join(OUTPUT_DIR, "val_data.json"), "w", encoding="utf-8") as f:
    json.dump(val, f, ensure_ascii=False, indent=2)

with open(os.path.join(OUTPUT_DIR, "test_data.json"), "w", encoding="utf-8") as f:
    json.dump(test, f, ensure_ascii=False, indent=2)

print("✅ 데이터 저장 완료!")


✅ 데이터 저장 완료!
